## Development notebook 
Walking through process intuitively 

In [2]:
# all of shakespeare's works, concatenated 
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-05-16 13:48:21--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2025-05-16 13:48:21 (7.82 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [3]:
with open('input.txt', 'r', encoding='utf-8') as f: 
    text = f.read() 

In [4]:
print("number of chars in dataset: ", len(text))

number of chars in dataset:  1115394


In [5]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [6]:
# get unique characters, sorted
# funny that 3 is the only number? 
chars = sorted(list(set(text))) 
vocab_size = len(chars) 
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
# mapping from characters to integers 
# OpenAI uses tiktoken 
stoi = {ch:i for i,ch in enumerate(chars)} # string to integer 
itos = {i:ch for i,ch in enumerate(chars)} # integer to string 

encode = lambda s: [stoi[c] for c in s] # encodes chars 
decode = lambda l: ''.join([itos[i] for i in l]) # decodes numbers 

In [8]:
print(encode("hellooo"))
print(decode(encode("hellooo")))

[46, 43, 50, 50, 53, 53, 53]
hellooo


In [9]:
import torch # pytorch. Need to use python3.8, doesn't support 3.13 

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [10]:
# train validation split 
n = int(0.9*len(data))
train = data[:n]
val = data[n:]

In [11]:
# We want transformer to be used to seeing any number of chars as input 
# Anything up to block size 
block_size = 8 # kind of arbitrary? 
x = train[:block_size]
y = train[1:block_size+1]
for t in range(block_size): 
    context = x[:t+1]   
    target = y[t] # y[t] = x[t+1] by definition. always 1 step ahead 
    print(f"when input is {context} the target is: {target}")

when input is tensor([18]) the target is: 47
when input is tensor([18, 47]) the target is: 56
when input is tensor([18, 47, 56]) the target is: 57
when input is tensor([18, 47, 56, 57]) the target is: 58
when input is tensor([18, 47, 56, 57, 58]) the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


In [12]:
# different way of writing the above 
block_size = 8 
sample = train[:block_size+1]
for t in range(block_size): 
    context = sample[:t+1]   
    target = sample[t+1]  
    print(f"when input is {context} the target is: {target}")

when input is tensor([18]) the target is: 47
when input is tensor([18, 47]) the target is: 56
when input is tensor([18, 47, 56]) the target is: 57
when input is tensor([18, 47, 56, 57]) the target is: 58
when input is tensor([18, 47, 56, 57, 58]) the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


In [13]:
torch.manual_seed(1337) 
batch_size = 4 # how many sequences we'll process in parallel 
block_size = 8 # max context length 

def get_batch(split): # split = train or val? 
    data = train if split == 'train' else val 
    ix = torch.randint(len(data) - block_size, (batch_size,)) # random offsets into training set
    x = torch.stack([data[i:i+block_size] for i in ix]) # stack = stack 1D tensors as rows
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x,y 

xb, yb = get_batch('train')

In [ ]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape) # 4 sets of length-8 text, randomly sampled from data
print(xb) 

print('targets:')
print(yb.shape) # same as the above but offset by 1 (+1) 
print(yb)

print('-----')
# all of our 32 training examples / observations
for b in range(batch_size): # rows 
    for t in range(block_size): # examples within rows 
        context = xb[b, :t+1]
        target = yb[b,t] # imo easier to remember yb[t] is the same as xb[t+1]
        print(f"when input is {context.tolist()} then target is {target}")


inputs:
torch.Size([4, 8])
tensor([[43,  1, 51, 39, 63,  1, 40, 43],
        [58, 46, 43,  1, 43, 39, 56, 57],
        [39, 58, 47, 53, 52, 12,  1, 37],
        [53, 56, 43,  1, 21,  1, 41, 39]])
targets:
torch.Size([4, 8])
tensor([[ 1, 51, 39, 63,  1, 40, 43,  1],
        [46, 43,  1, 43, 39, 56, 57, 10],
        [58, 47, 53, 52, 12,  1, 37, 53],
        [56, 43,  1, 21,  1, 41, 39, 51]])
-----
when input is [43] then target is 1
when input is [43, 1] then target is 51
when input is [43, 1, 51] then target is 39
when input is [43, 1, 51, 39] then target is 63
when input is [43, 1, 51, 39, 63] then target is 1
when input is [43, 1, 51, 39, 63, 1] then target is 40
when input is [43, 1, 51, 39, 63, 1, 40] then target is 43
when input is [43, 1, 51, 39, 63, 1, 40, 43] then target is 1
when input is [58] then target is 46
when input is [58, 46] then target is 43
when input is [58, 46, 43] then target is 1
when input is [58, 46, 43, 1] then target is 43
when input is [58, 46, 43, 1, 43] th